In [1]:
from table_corrigé2 import *
from corde import *
from excitation import exc_ref
import fonctions_ss as ss
from variation_param import param_variation
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Audio
%matplotlib widget
plt.rcParams["figure.figsize"] = (12,4)
from tqdm import tqdm
from scipy.io import wavfile
import pickle as pkl

In [2]:
#Choix de la configuration de la guitare
table = medium_1
corde = corde_acier_1
Fe = 2**14

#Emplacement des fichiers d'enregistrement
path = "../../../Dataset/Test_vico2/Medium_1/Corde_acier_1/"

#Génération de variations autour de ces configurations de référence
N_sample = 500
_, table_dicos, corde_dicos, exc_dicos, noise_prop = param_variation(N_sample,table,corde, exc_ref)

norme = 50000 #a l'arrache

dico = {}
for i in tqdm(range(N_sample)) :
    table_dico = table_dicos[i]
    corde_dico = corde_dicos[i]
    exc_dico = exc_dicos[i]
    acc_Nt = ss.Main_ss(table_dico,corde_dico, exc_ref=exc_dico, Fe=Fe, obs="force", plot_fext=False, plot_deformee=False, debug=False)
    # acc_Nt /= np.max(np.abs(acc_Nt))
    acc_Nt /= norme
    if np.max(np.abs(acc_Nt)) > 1 :
        print(f"Attention, le signal {'acc_Nt_' + str(i) + '.wav'} clip")
    
    wavfile.write(path + "acc_Nt_" + str(i) + ".wav", Fe, acc_Nt)

    #Ajout d'un bruit blanc tel que la variance du rapport signal bruit soit de 8.39dB. On veut également que le rapport signal bruit soit de 17.75db
    # acc_Nt += noise_prop[i]*np.random.normal(0, 10**(-8.39/20), acc_Nt.shape)
    # acc_Nt += 10**(-np.random.normal(17.75, 8.39)/20)*np.random.normal(0, 1, acc_Nt.shape)
    # display(Audio(acc_Nt, rate=Fe))

    kle = str(i) + ".wav"
    dico[kle] = {}
    dico[kle]['acc_Nt'] = acc_Nt
    dico[kle]['T'] = corde_dico['T']
    dico[kle]['rho_l'] = corde_dico['rho_l']
    dico[kle]['Lc'] = corde_dico['Lc']
    dico[kle]['B'] = corde_dico['B']
    dico[kle]['rho_T'] = table_dico['rho_T']
    dico[kle]['L_x'] = table_dico['L_x']
    dico[kle]['L_y'] = table_dico['L_y']
    dico[kle]['h'] = table_dico['h']
    dico[kle]['E_nu'] = table_dico['E_nu']
    dico[kle]['xexc_ratio'] = exc_dico['xexc_ratio']
    dico[kle]['temps_desc'] = exc_dico['temps_desc']
    dico[kle]['Fe'] = Fe

#Saving the big dictionnary
with open(path + "details.pkl", "wb") as f:
    pkl.dump(dico, f)


100%|██████████| 500/500 [21:53<00:00,  2.63s/it]


In [3]:
#Choix de la configuration de la guitare
table = plexi
corde = corde_acier_1
Fe = 2**14

#Emplacement des fichiers d'enregistrement
path = "../../../Dataset/Test_vico2/Plexi/Corde_acier_1/"

#Génération de variations autour de ces configurations de référence
N_sample = 1000
_, table_dicos, corde_dicos, exc_dicos, noise_prop = param_variation(N_sample,table,corde, exc_ref)

dico = {}
for i in tqdm(range(N_sample)) :
    table_dico = table_dicos[i]
    corde_dico = corde_dicos[i]
    exc_dico = exc_dicos[i]
    acc_Nt = ss.Main_ss(table_dico,corde_dico, exc_ref=exc_dico, Fe=Fe, obs="force", plot_fext=False, plot_deformee=False, debug=False)
    # acc_Nt /= np.max(np.abs(acc_Nt))
    wavfile.write(path + "acc_Nt_" + str(i) + ".wav", Fe, acc_Nt.astype(np.int32))
    #Ajout d'un bruit blanc tel que la variance du rapport signal bruit soit de 8.39dB. On veut également que le rapport signal bruit soit de 17.75db
    # acc_Nt += noise_prop[i]*np.random.normal(0, 10**(-8.39/20), acc_Nt.shape)
    # acc_Nt += 10**(-np.random.normal(17.75, 8.39)/20)*np.random.normal(0, 1, acc_Nt.shape)
    # display(Audio(acc_Nt, rate=Fe))

    kle = str(i) + ".wav"
    dico[kle] = {}
    dico[kle]['acc_Nt'] = acc_Nt
    dico[kle]['T'] = corde_dico['T']
    dico[kle]['rho_l'] = corde_dico['rho_l']
    dico[kle]['Lc'] = corde_dico['Lc']
    dico[kle]['B'] = corde_dico['B']
    dico[kle]['rho_T'] = table_dico['rho_T']
    dico[kle]['L_x'] = table_dico['L_x']
    dico[kle]['L_y'] = table_dico['L_y']
    dico[kle]['h'] = table_dico['h']
    dico[kle]['E_nu'] = table_dico['E_nu']
    dico[kle]['xexc_ratio'] = exc_dico['xexc_ratio']
    dico[kle]['temps_desc'] = exc_dico['temps_desc']
    dico[kle]['Fe'] = Fe

#Saving the big dictionnary
with open(path + "details.pkl", "wb") as f:
    pkl.dump(dico, f)


  0%|          | 0/1000 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
np.max(np.abs(acc_Nt))

In [ ]:
#Generate a pink noise
from scipy.signal import lfilter, butter
from numpy.random import randn
def pink_noise(n_samples, f_sample):
    b, a = butter(5, [0.5, 1000], btype='bandpass', fs=f_sample, output='ba')
    pink_noise = lfilter(b, a, randn(n_samples))
    return pink_noise

#Add the pink noise to the signal
acc_Nt += 10**(-np.random.normal(17.75, 8.39)/20)*pink_noise(acc_Nt.shape[0], Fe)
display(Audio(acc_Nt, rate=Fe))